In [4]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [5]:
import json
def get_current_weather(location:str, unit:str="fahrenheit")->str:
    if "tokyo" in location.lower():
        return json.dumps({"location": "tokyo", "temperature":"10","unit":"celsius"})
    elif "francisco" in location.lower():
        return json.dumps({"location": "francisco", "temperature":"76","unit":"fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "paris", "temperature":"23","unit":"celsius"})
    else:
        return json.dumps({"location": location,"unit":"unknown"})
    


In [10]:
from openai.types.chat.chat_completion import ChatCompletion, ChatCompletionMessage
def run_conversation(main_request:str)->str:
    messages = [{"role":"user","content":main_request}]
    tools = [
        {
            "type":"function",
            "function":{
                "name":"get_current_weather",
                "description":"Get the current weather in the given location",
                "parameters":{
                    "type" :"object",
                    "properties":{
                        "location":{
                            "type":"string",
                            "description":"The city and state; like paris and CA",
                        },
                        "unit":{"type":"string","enum":["celsius", "fahrenheit"]},

                    },
                    "required":["location"]
                },
            },
        },
    ]

     # First Request
    response:ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages = messages,
        tools=tools,
        tool_choice="auto"
    )
  

    response_message:ChatCompletionMessage = response.choices[0].message
    print("first response : ", dict(response_message))

    tools_call = response_message.tool_calls
    display("first rsponse tools-calls is :", list(tools_call))


    # Step 2: check if the model wanted to call a function

    if tools_call:
        available_functions = {
            "get_current_weather" :get_current_weather,
        }

        messages.append(response_message)

        for toll_call in tools_call:
            function_name = toll_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(toll_call.function.arguments)
            function_response = function_to_call(
                location = function_args.get("location"),
                unit = function_args.get("unit")
            )
            messages.append(
                {
                    "tool_call_id": toll_call.id,
                    "role":"tool",
                    "name": function_name,
                    "content":function_response,

                }
            )


        display("second Request Message :", list(messages))

        second_response :ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages = messages,
        )

        display("second response :", dict(second_response))

        return second_response.choices[0].message.content






run_conversation("What's the weather like in San Francisco, Tokyo, and Paris?")


first response :  {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [ChatCompletionMessageToolCall(id='call_iTDdJG6Fl2frAcOWitlYNntk', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_WgYSX8TGYu88JbM25GKZImUX', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_1rBiwahFR9Zspe3EGhhQECe3', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]}


'first rsponse tools-calls is :'

[ChatCompletionMessageToolCall(id='call_iTDdJG6Fl2frAcOWitlYNntk', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_WgYSX8TGYu88JbM25GKZImUX', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_1rBiwahFR9Zspe3EGhhQECe3', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]

'second Request Message :'

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_iTDdJG6Fl2frAcOWitlYNntk', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_WgYSX8TGYu88JbM25GKZImUX', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_1rBiwahFR9Zspe3EGhhQECe3', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_iTDdJG6Fl2frAcOWitlYNntk',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "francisco", "temperature": "76", "unit": "fahrenheit"}'},
 {'tool_call_id': 'call_WgYSX8TGYu88JbM25GKZImUX',
  'role': 'too

'second response :'

{'id': 'chatcmpl-8SqoNNy5bjs7YpFRUqyXOBcXv71Lj',
 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Currently, the weather in San Francisco is 76°F, in Tokyo it's 10°C, and in Paris it's 23°C.", role='assistant', function_call=None, tool_calls=None))],
 'created': 1701886883,
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_eeff13170a',
 'usage': CompletionUsage(completion_tokens=29, prompt_tokens=171, total_tokens=200)}

"Currently, the weather in San Francisco is 76°F, in Tokyo it's 10°C, and in Paris it's 23°C."